#Project name

#1. Stop and Search Data
##1.1 Download stop and search data and combine into single file
The data comes from:
For the purpose of this project, we downloaded stop and search for the year 2019 -- Jan, 1 to Dec, 31. 
The source files were monthly reports - we combine them below. 

In [1]:
import pandas as pd
import geopandas as gpd
import glob 
import json

In [3]:
files = glob.glob("/Users/sean/desktop/proj_london/data/source_data/*.csv")
appended = [pd.read_csv(f) for f in files]

In [3]:
combined = pd.concat(appended, ignore_index=True)
combined.to_csv(r'/Users/sean/desktop/proj_london/data/combined_stop_data.csv')

NameError: name 'pd' is not defined

##1.2 Read combined stop and search data 
Read combined data csv and check data:

In [13]:
df = pd.read_csv('combined_data.csv', dtype={"Latitude": str, "Longitude": str})

/opt/anaconda3/envs/london-data/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
df.head()

,Unnamed: 0,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing
0,0,Person and Vehicle search,2019-07-31T23:00:00+00:00,0.0,NaN,51.460887,0.005072,Male,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,NaN
1,1,Person and Vehicle search,2019-07-31T23:02:00+00:00,0.0,NaN,51.480146999999995,-0.507249,Male,25-34,Black/African/Caribbean/Black British - Any ot...,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest,NaN,NaN
2,2,Person search,2019-07-31T23:02:00+00:00,0.0,NaN,51.501283,-0.224885,Male,NaN,Other ethnic group - Not stated,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,NaN
3,3,Person search,2019-07-31T23:02:00+00:00,0.0,NaN,51.501283,-0.224885,Female,NaN,Other ethnic group - Not stated,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,NaN
4,4,Person search,2019-07-31T23:05:00+00:00,0.0,NaN,51.486675,-0.093976,Male,25-34,Asian/Asian British - Any other Asian background,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,NaN


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268469 entries, 0 to 268468
Data columns (total 16 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Unnamed: 0                                268469 non-null  int64  
 1   Type                                      268469 non-null  object 
 2   Date                                      268469 non-null  object 
 3   Part of a policing operation              265940 non-null  float64
 4   Policing operation                        0 non-null       float64
 5   Latitude                                  241403 non-null  object 
 6   Longitude                                 241403 non-null  object 
 7   Gender                                    264904 non-null  object 
 8   Age range                                 239826 non-null  object 
 9   Self-defined ethnicity                    264774 non-null  object 
 10  Officer-defined ethn

##1.3 Clean dataframe
Change column names and drop irrelevant columns:

In [17]:
df.rename(columns=
         {'Self-defined ethnicity': 'ethnicity',
         'Officer-defined ethnicity':'officer_ethnicity',
         'Object of search': 'purpose',
         'Part of a policing operation': 'operation',
         'Age range': 'age_range',
         'Gender': 'gender',
         'Outcome': 'outcome',
         'Legislation': 'legislation',
         'Date': 'date',
          'Latitude': 'latitude',
          'Longitude': 'longitude',
          'Type': 'type'
         },
         inplace=True)
df.columns

Index(['Unnamed: 0', 'type', 'date', 'operation', 'Policing operation',
       'latitude', 'longitude', 'gender', 'age_range', 'ethnicity',
       'officer_ethnicity', 'legislation', 'purpose', 'outcome',
       'Outcome linked to object of search',
       'Removal of more than just outer clothing'],
      dtype='object')

In [18]:
df = df.drop(['Policing operation', 'Outcome linked to object of search', 'Removal of more than just outer clothing'], axis = 1) 

In [19]:
df_cleaned = df.dropna(axis=0)

In [20]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212165 entries, 0 to 268315
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         212165 non-null  int64  
 1   type               212165 non-null  object 
 2   date               212165 non-null  object 
 3   operation          212165 non-null  float64
 4   latitude           212165 non-null  object 
 5   longitude          212165 non-null  object 
 6   gender             212165 non-null  object 
 7   age_range          212165 non-null  object 
 8   ethnicity          212165 non-null  object 
 9   officer_ethnicity  212165 non-null  object 
 10  legislation        212165 non-null  object 
 11  purpose            212165 non-null  object 
 12  outcome            212165 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 22.7+ MB


##1.4 Add coordinate column
Combine lat and long to create coordinate column for easy referencing

In [21]:
df_cleaned['coord'] = "(" + df_cleaned.Latitude + ", " + df_cleaned.Longitude + ")" 

AttributeError: 'DataFrame' object has no attribute 'Latitude'

In [22]:
df_cleaned.head()

,Unnamed: 0,type,date,operation,latitude,longitude,gender,age_range,ethnicity,officer_ethnicity,legislation,purpose,outcome
0,0,Person and Vehicle search,2019-07-31T23:00:00+00:00,0.0,51.460887,0.005072,Male,25-34,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal
1,1,Person and Vehicle search,2019-07-31T23:02:00+00:00,0.0,51.480146999999995,-0.507249,Male,25-34,Black/African/Caribbean/Black British - Any ot...,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest
4,4,Person search,2019-07-31T23:05:00+00:00,0.0,51.486675,-0.093976,Male,25-34,Asian/Asian British - Any other Asian background,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal
5,5,Person search,2019-07-31T23:05:00+00:00,0.0,51.509962,-0.131179,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution
8,8,Person search,2019-07-31T23:07:00+00:00,0.0,51.509962,-0.131179,Male,18-24,Black/African/Caribbean/Black British - Caribbean,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution


In [23]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212165 entries, 0 to 268315
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         212165 non-null  int64  
 1   type               212165 non-null  object 
 2   date               212165 non-null  object 
 3   operation          212165 non-null  float64
 4   latitude           212165 non-null  object 
 5   longitude          212165 non-null  object 
 6   gender             212165 non-null  object 
 7   age_range          212165 non-null  object 
 8   ethnicity          212165 non-null  object 
 9   officer_ethnicity  212165 non-null  object 
 10  legislation        212165 non-null  object 
 11  purpose            212165 non-null  object 
 12  outcome            212165 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 22.7+ MB


In [2]:
df_cleaned.to_csv(r'/Users/sean/Desktop/proj_london/data/cleaned_stop_data.csv')

NameError: name 'df_cleaned' is not defined

#2 Reshape London GeoJSON
##2.1 Download London GeoJSON and add ID to each feature